In [142]:
import math
import numpy as np

class Cluster:
    '''初始化'''
    def __init__(self, X, y, base=10):
        self._X, self._y = X, y
        self.counters = np.bincount(self._y)
        self._con_chaose_cache = self._ent_cache = None
        self.base = base
        
    def ent(self):
        '''计算信息熵'''
        _ent = self.counters    # (关键)计算每个类的数量
        _len = len(self._y)
        _ent_cache = -sum([_c/_len * math.log(_c/_len, self.base) if _c != 0 else 0 for _c in _ent])
        return  _ent_cache
    
    def con_chaos(self, idx):
        '''计算某一特征条件熵—H(y!A)'''
        data = self._X.T[idx]                                         # 选取某一特征    
        features = set(data)                                          # 特征的标签，顺次标记
   
        tmp_labels = [data == feature for feature in features]       #（非常重要—核心）某一特征某个取值的空间坐标，用(..., True,False,...)表示
        _con_chaos_cache = [np.sum(_label) for _label in tmp_labels] # 统计某一特征的不同取值所占有的数量,用于算增益比(先忽略)
        label_lst = [self._y[label] for label in tmp_labels]         # (非常重要)某一特征取值对应的类y输出，用于计算每个取值的信息熵
        
        rs, chaos_lst = 0, []
        for data_label, tar_label in zip(tmp_labels, label_lst):       
            tmp_data = self._X[data_label]
            
            _chaos = Cluster(tmp_data, tar_label, base=self.base).ent()  # 计算不同取值的条件熵
            
            rs += len(tmp_data) / len(data) * _chaos                     # 加权和
            chaos_lst.append(_chaos)        
            #print(str(tmp_data) + '—对应—' + str(tar_label) + '\n')  
        #print('特征【' + str(idx) + '】取值:'+ str(features) + '所对应的信息熵：' + str(chaos_lst))
        return rs, chaos_lst  
    
    def info_gain(self, idx):
        _con_chaos, _chaos_lst = self.con_chaos(idx)
        _gain = self.ent() - _con_chaos
        return _gain
    
    def bin_con_chaos(self, idx, tar, continues=False):
        '''转化为二分类问题'''
        data = self._X.T[idx]
        tar = data == tar if not continues else data < tar    #转化为二分类的条件，如果True，tar可以为任意数
        tmp_labels = [tar, ~tar]

        _con_chaos_cache = [np.sum(_label) for _label in tmp_labels]  
        label_lst = [self._y[label] for label in tmp_labels]
    
        rs, chaos_lst = 0, []
        for data_label, tar_label in zip(tmp_labels, label_lst):       
            tmp_data = self._X[data_label]
            
            _chaos = Cluster(tmp_data, tar_label, base=self.base).ent()
            
            rs += len(tmp_data) / len(data) * _chaos  
            chaos_lst.append(_chaos)        
            #print(str(tmp_data) + '—对应—' + str(tar_label) + '\n')  
        #print('某一特征不同取值'+ '对应的信息熵：' + str(chaos_lst))
        return rs, chaos_lst
    
    def bin_info_gain(self, idx, tar, continues=False):
        _con_chaos, _chaos_lst = self.bin_con_chaos(idx, tar, continues)
        _gain = self.ent() - _con_chaos
        return _gain
    
_X = np.random.randint(3, size=(10,2))
_y = np.random.randint(3, size=10)

#_X = np.array([[2, 0], [1, 0], [2, 2], [2, 1], [1, 0], [1, 0], [1, 1], [2, 1], [0, 1], [2, 0]])
#_y = np.array([2, 1, 0, 0, 0, 2, 0, 2, 0, 0])

Cls = Cluster(_X, _y)
_ent = Cls.ent()
_idx = 0   # 选择一个特征维度
_tar = 1   # 对于二分类问题，选择一个任意数（如果 continues = True）
print('类的信息熵： ' + str(_ent))
print('_' * 100)

_con_chaos, _con_chaos_lst = Cls.con_chaos(_idx)
print('条件熵： ' + str(_con_chaos) + ' ......包含： ' + str(_con_chaos_lst) + '\n')

_gin = Cls.info_gain(_idx)
print('信息增益： ' + str(_gin))
print('_' * 100)

_bin_con_chaos, _bin_chaos_lst = Cls.bin_con_chaos(_idx, _tar, continues=False)
print('二分类问题条件熵： ' + str(_bin_con_chaos) + ' ......包含： ' + str(_bin_chaos_lst) + '\n')

_bin_gin = Cls.bin_info_gain(_idx, _tar, continues=False)
print('二分类问题信息增益： ' + str(_bin_gin))

类的信息熵： 0.412697251504
____________________________________________________________________________________________________
条件熵： 0.263548374681 ......包含： [0.27643459094367495, 0.24421905028821553, 0.27643459094367495]

信息增益： 0.149148876823
____________________________________________________________________________________________________
二分类问题条件熵： 0.361235994797 ......包含： [0.24421905028821553, 0.43924729113581862]

二分类问题信息增益： 0.0514612567074
